In [2]:
!pip install googletrans==3.1.0a0 transformers sentencepiece -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [3]:
!pip install gdown

In [4]:
!gdown --id 199Eo_tf8_7U-CPjVm74F4ijk-8lahpQh

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=199Eo_tf8_7U-CPjVm74F4ijk-8lahpQh
To: /kaggle/working/goemotions_subset_train_new_part1.csv
100%|███████████████████████████████████████| 3.44M/3.44M [00:00<00:00, 173MB/s]


In [5]:
import pandas as pd
import numpy as np
import re
from googletrans import Translator
translator = Translator()

In [6]:
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/MultiIndicParaphraseGeneration")
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/MultiIndicParaphraseGeneration")

# Some initial mapping
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")

In [8]:
inp = tokenizer("दिल्ली यूनिवर्सिटी देश की प्रसिद्ध यूनिवर्सिटी में से एक है. </s> <2hi>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids 

model_output=model.generate(inp, use_cache=True,no_repeat_ngram_size=3,encoder_no_repeat_ngram_size=3, num_beams=4, max_length=40, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2hi>"))

# Decode to get output strings
decoded_output=tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(decoded_output) # दिल्ली विश्वविद्यालय देश की प्रमुख विश्वविद्यालयों में शामिल है।

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
दिल्ली विश्वविद्यालय देश की प्रमुख विश्वविद्यालयों में शामिल है।


In [9]:
df = pd.read_csv('goemotions_subset_train_new_part1.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,text,three_point,marathi_text
0,0,0,13178,"This is so hard to sit through, I can't believ...",-1,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व..."
1,1,1,39432,Because idiots in this sub care too much about...,-1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...
2,3,3,30127,Water is not a human rights. That's absurd.,-1,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.
3,4,4,10283,The thin side and endlines are really throwing...,-1,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.
4,5,5,2930,If there is no goal on this powerplay it will ...,-1,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...


In [10]:
df.drop(['Unnamed: 0.1.1', 'Unnamed: 0.1', 'Unnamed: 0', 'text'], axis=1, inplace=True)
dict1 = {'marathi_text': 'tweet', 'three_point': 'label'}
df.rename(columns=dict1, inplace=True)
df = df.iloc[:, [1, 0]]

In [11]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1


In [12]:
# for paraphrasing, create a seperate 'paraphrase' column
df['paraphrased'] = ""

In [13]:
df.head()

,tweet,label,paraphrased
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1,
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1,
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1,
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1,
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1,


In [14]:
# data preprocessing:
# removing non marathi text 
import re
def remove_non_marathi_goemotions(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    # i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [15]:
df['tweet'][205]

'[NAME] [NAME] हा कन्सोलच्या 72 तासांसारखा आहे अर्थातच आम्हाला तो wtf<3 आवडतो'

In [16]:
remove_non_marathi_goemotions(df['tweet'][205])

'हा कन्सोलच्या तासांसारखा आहे अर्थातच आम्हाला तो < आवडतो'

In [17]:
for i in range(len(df)):
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])
df = df[df['tweet']!='']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [19]:
df['tweet'][3054]

'नाम, काय? तू इथेच का आहेस. बंद संभोग.'

In [21]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
  df['tweet'][i] = remove_non_marathi_goemotions(df['tweet'][i])
  inp = tokenizer(df['tweet'][i]+" </s> <2mr>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids 

  model_output=model.generate(inp, use_cache=True,no_repeat_ngram_size=3,encoder_no_repeat_ngram_size=3, num_beams=4, max_length=40, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2mr>"))

  # Decode to get output strings
  decoded_output=tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
  df['paraphrased'][i] = decoded_output

  0%|          | 0/12491 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 12491/12491 [7:01:48<00:00,  2.03s/it]  


In [22]:
# once dataframe is ready, then create a new dataframe, containing the 'paraphrased' and 'label' columns only:
df_paraphrased = df[['paraphrased', 'label']]
df_paraphrased.head()

,paraphrased,label
0,हे बसणं कठीण आहे. मला या डुडच्या प्रतिज्ञांवर ...,-1
1,कारण या चित्रपटातले इडियट इतर लोकांच्या स्क्रि...,-1
2,पाणी हा मानवाचा अधिकार नाही.,-1
3,पातळ बाजू व अंतरीषा खरोखरच माझ्यावर फेकून देता...,-1
4,असे न झाल्यास निराश होणे कठीण आहे.,-1


In [25]:
df_paraphrased['paraphrased'][1]

'कारण या चित्रपटातले इडियट इतर लोकांच्या स्क्रिनवर काय असते याची काळजी घेतात. हे फक्त माझ्या मते आहे की इथे बरेच लोक डॉक्टर आहेत.'

In [26]:
df['tweet'][1]

'कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.'

In [27]:
df_paraphrased.rename(columns = {'paraphrased':'tweet'}, inplace = True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
df.head()

,tweet,label,paraphrased
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1,हे बसणं कठीण आहे. मला या डुडच्या प्रतिज्ञांवर ...
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1,कारण या चित्रपटातले इडियट इतर लोकांच्या स्क्रि...
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1,पाणी हा मानवाचा अधिकार नाही.
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1,पातळ बाजू व अंतरीषा खरोखरच माझ्यावर फेकून देता...
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1,असे न झाल्यास निराश होणे कठीण आहे.


In [29]:
df.drop(['paraphrased'], axis=1, inplace=True)

In [30]:
df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1


In [31]:
len(df_paraphrased)

12491

In [32]:
df_paraphrased.drop_duplicates()

,tweet,label
0,हे बसणं कठीण आहे. मला या डुडच्या प्रतिज्ञांवर ...,-1
1,कारण या चित्रपटातले इडियट इतर लोकांच्या स्क्रि...,-1
2,पाणी हा मानवाचा अधिकार नाही.,-1
3,पातळ बाजू व अंतरीषा खरोखरच माझ्यावर फेकून देता...,-1
4,असे न झाल्यास निराश होणे कठीण आहे.,-1
...,...,...
12486,स्त्रीला पुरुषांप्रमाणे सर्व अधिकार मिळायला हवेत.,1
12487,माझ्याकडे वास्तवानंतर काही मोठे आत्मनिरीक्षण आहे.,1
12488,बुडणाऱ्या शाळा बससारखी मजेदार.,1
12489,"उत्तम ठिकाणे, स्थलांतरित हॉटेल्स, स्वस्त आणि च...",1


In [33]:
df_paraphrased['tweet'][1]

'कारण या चित्रपटातले इडियट इतर लोकांच्या स्क्रिनवर काय असते याची काळजी घेतात. हे फक्त माझ्या मते आहे की इथे बरेच लोक डॉक्टर आहेत.'

In [34]:
df['tweet'][1]

'कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर काय आहे याची खूप काळजी घेतात? हे फक्त मला सांगते की येथे बरेच लोक नार्सिसिस्ट आहेत.'

In [35]:
df_final = df.append(df_paraphrased, ignore_index = True)

In [36]:
df_final.to_csv('augmented-with-paraphrasing-goemotions.csv', index=False)
# use this dataset directly for training. do not preprocess this at all.

In [37]:
df_final

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1
...,...,...
24977,स्त्रीला पुरुषांप्रमाणे सर्व अधिकार मिळायला हवेत.,1
24978,माझ्याकडे वास्तवानंतर काही मोठे आत्मनिरीक्षण आहे.,1
24979,बुडणाऱ्या शाळा बससारखी मजेदार.,1
24980,"उत्तम ठिकाणे, स्थलांतरित हॉटेल्स, स्वस्त आणि च...",1


In [38]:
df_final['tweet'][0]

'हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर विश्वास बसत नाही, हे सर्व अधिक असह्य करते'

In [39]:
df_final['tweet'][12491]

'हे बसणं कठीण आहे. मला या डुडच्या प्रतिज्ञांवर विश्वास बसतो नाही. हे सर्व असह्य करते.'

In [40]:
len(df_final)

24982